In [1]:
!nvidia-smi

Wed Oct 11 16:43:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.191.01   Driver Version: 450.191.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   32C    P0    60W / 300W |  29704MiB / 32510MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [5]:
from datasets import Dataset, DatasetDict
from transformers import T5ForConditionalGeneration, T5Config, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import datasets
from typing import Optional
from transformers.modeling_outputs import Seq2SeqLMOutput
from transformers.trainer import is_datasets_available, seed_worker
from datasets import load_dataset

In [6]:
from datasets import load_dataset
tokenized_sur = load_dataset("carnival13/test_DA_tokenized2")

In [7]:
# model_ckpt = "massive_da_eng_sur_rt5/final"
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [8]:
class RankT5(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
        config.rank_score_index = 32019
        config.n_pass = 10
        config.output_hidden_states = False
        super().__init__(config)
#         self.rank_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.rank_id = config.rank_score_index
        self.n_pass = config.n_pass

    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, pass_label=None, **kwargs):

        batch_size_n, seq_len = input_ids.size()
        batch_size = int(batch_size_n/self.n_pass)
        labels = None

        # input_ids = input_ids.view(batch_size*n_pass, -1)
        # attention_mask = attention_mask.view(batch_size*n_pass, -1)
        if decoder_input_ids == None and labels == None:
            decoder_input_ids = torch.zeros((batch_size_n, 1), dtype=int).to(input_ids.device)
        
        
        
        if labels != None and decoder_input_ids == None:
#             batch_size, decoder_seq_len = labels.size()
#             labels = labels.view(batch_size, 1, decoder_seq_len).contiguous()
#             labels = labels.expand(batch_size, n_pass, decoder_seq_len).contiguous()

#             labels = labels.view(batch_size*n_pass, -1)
            decoder_input_ids = self._shift_right(labels)
            print(decoder_input_ids.size())


        out = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
#         rank_score = self.rank_head(out.decoder_hidden_states[-1][:, 0, :])
        rank_score = out.logits[:, 0, :] 
        out.rank_score = rank_score[:, self.rank_id].view(-1, self.n_pass)
        loss = None
        assert out.decoder_hidden_states == None

        if pass_label != None:
            pass_label = pass_label[::self.n_pass]
            rank_score = out.rank_score
#             gen_score = out.gpe_score

            loss_fct1 = nn.CrossEntropyLoss()
#             loss_fct2 = nn.CrossEntropyLoss()

            loss = loss_fct1(rank_score, pass_label.view(-1))

        ret =  Seq2SeqLMOutput(
            loss=loss,
            logits=out.logits,
            past_key_values=out.past_key_values,
            decoder_hidden_states=out.decoder_hidden_states,
            decoder_attentions=out.decoder_attentions,
            cross_attentions=out.cross_attentions,
            encoder_last_hidden_state=out.encoder_last_hidden_state,
            encoder_hidden_states=out.encoder_hidden_states,
            encoder_attentions=out.encoder_attentions,
        )
        ret.rank_score = out.rank_score
        return ret
        
    
model_ckpt = "massive_da_eng_sur_rt5/final"
model = RankT5.from_pretrained(model_ckpt)

from transformers import AutoTokenizer
mod_ckp = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(mod_ckp)
question = "query:"
title = "intent:"
context = "examples:"
eou = "<eou>"
tokenizer.add_tokens([question, title, context, eou], special_tokens=True)

/raid/kavin-intern-maunendra/anaconda3/envs/wow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


4

In [9]:
model = RankT5.from_pretrained(model_ckpt)

In [10]:
# from huggingface_hub import notebook_login
# notebook_login()

In [11]:
# model.push_to_hub(model_ckpt)

In [12]:
device = "cuda:5" if torch.cuda.is_available() else "cpu"

In [13]:
device

'cuda:5'

In [14]:
model_ckpt = "massive_da_eng4/final"
tokenizer2 = AutoTokenizer.from_pretrained(model_ckpt)

In [19]:
def decode(df):
    return {"input": tokenizer2.batch_decode(df["input_ids"], skip_special_tokens=True)}

In [24]:
utk = tokenized_sur.map(decode, batched=True, remove_columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/335850 [00:00<?, ? examples/s]

In [25]:
def tokenize(df):
    return tokenizer(df["input"], padding=False, truncation=True, max_length=512)

In [28]:
tokenized_sur = utk.map(tokenize, batched=True, remove_columns=["input"])

Map:   0%|          | 0/335850 [00:00<?, ? examples/s]

In [29]:
tokenized_sur.set_format(type="torch", columns=["input_ids", "attention_mask", "pass_label"])

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer)
dl = DataLoader(tokenized_sur["train"], collate_fn=data_collator, batch_size=60, shuffle=False)

In [31]:
from tqdm.notebook import tqdm
model = model.to(device)
batch_len = []
labels = []
scores = []

model.eval()
with torch.no_grad():
    for b in tqdm(dl):
        batch_len.append(int(b["input_ids"].size()[0]))
        input_ids = b["input_ids"].to(device)
        attention_mask = b["attention_mask"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        rank_score = list(outputs.rank_score.view(-1))
        labels += list(b["pass_label"])
        scores += rank_score
        
#         print(pass_label.size())
#         mrr.append(cal_mrr(rank_score, pass_label))
#         if pred == None:
#             pred = rank_score
#             lab_p = pass_label
#         else:
#             pred = torch.cat((pred, rank_score), 0)
#             lab_p = torch.cat((lab_p, pass_label), 0)

#         for i in r_k.keys():
#             r_k[i].append(r_at_k(rank_score, pass_label, i))

  0%|          | 0/5598 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 720.00 MiB (GPU 5; 31.75 GiB total capacity; 17.35 GiB already allocated; 300.69 MiB free; 28.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
scores = torch.tensor(scores)
labels = torch.tensor(labels[::150])

In [ ]:
scores = scores.view(-1, 150)

In [19]:
scores.shape

torch.Size([2239, 150])

In [20]:
prediction = torch.argmax(scores, dim=1)

In [21]:
dct = {"id": [], "intent": []}
for i, j in enumerate(list(prediction)):
    dct["id"].append(i)
    dct["intent"].append(j)


In [22]:
def dict_to_dataset(dict):
    dataset = datasets.Dataset.from_dict(dict)
    return dataset

pred = dict_to_dataset(dct)
pred.save_to_disk(f"pred_xlmr")

Saving the dataset (0/1 shards):   0%|          | 0/2239 [00:00<?, ? examples/s]

In [23]:
acc_lst = labels == prediction

In [24]:
print(f"Accuracy = {acc_lst.sum()/2239}")

Accuracy = 0.8311746120452881


In [43]:
labels.view(-1, 1) in torch.argsort(scores, dim=1)[:, :10]

True

In [46]:
prediction

tensor([101, 109,   3,  ...,  47,   6,   3])